### Running in Docker container on Swoose

Started Docker container with the following command:

```docker run -p 8888:8888 -v /home/sam/data/pacbio_oly/:/home/data -it bioinformatics/bioinformatics:v0 /bin/bash```

The command allows ```/home/sam/data/pacbio_oly/``` to be accessible to the Docker container.

Once  access to Jupyter Notebook over port 8888 and makes my Jupyter Notebook GitHub repo and my data files the container was started, started Jupyter Notebook with the following command inside the Docker container:

```jupyter notebook --allow-root```

This is configured in the Docker container to launch a Jupyter Notebook without a browser on port 8888.
The Docker container is running on an image created from this [Dockerfile (Git commit db26291a](https://github.com/RobertsLab/code/commit/db26291a69d909ff41caa26a479b927ea6e501c8)

In [1]:
%%bash
date

Mon Sep 18 18:15:36 UTC 2017


In [2]:
%%bash
hostname

3291cf2bcb28


In [3]:
%%bash
lscpu

Architecture:          x86_64
CPU op-mode(s):        32-bit, 64-bit
Byte Order:            Little Endian
CPU(s):                24
On-line CPU(s) list:   0-23
Thread(s) per core:    2
Core(s) per socket:    6
Socket(s):             2
NUMA node(s):          1
Vendor ID:             GenuineIntel
CPU family:            6
Model:                 44
Model name:            Intel(R) Xeon(R) CPU           X5670  @ 2.93GHz
Stepping:              2
CPU MHz:               2926.129
BogoMIPS:              5851.98
Virtualization:        VT-x
L1d cache:             32K
L1i cache:             32K
L2 cache:              256K
L3 cache:              12288K
NUMA node0 CPU(s):     0-23


In [4]:
%%bash
free -mh

             total       used       free     shared    buffers     cached
Mem:           70G        68G       2.1G       232M       550M        61G
-/+ buffers/cache:       7.1G        63G
Swap:         4.7G       5.4M       4.7G


In [5]:
%%bash
pwd

/home/data


### Run [Racon v0.5.0](https://github.com/isovic/racon)

#### Racon is an "ultrafast consensus module for raw de novo genome assembly of long uncorrected reads."

#### Using as part of pipeline: minimap/miniasm/racon

In [7]:
%%bash
which racon

In [8]:
%%bash
/usr/local/bioinformatics/racon-v0.5.0/bin/racon

  /usr/local/bioinformatics/racon-v0.5.0/bin/racon [options] <reads.fastq> <overlaps.paf> <raw_contigs.fasta> <out_consensus.fasta>

Options
  Input/Output options:
    reads               STR   Reads in FASTQ format.
    alnpath             STR   Path to a MHAP file with read-to-target overlaps.
    raw                 STR   Path to the raw contig/read sequences (output from the layout step). GFA, FASTA/FASTQ or
                              SAM formats allowed.
    out                 STR   Output consensus sequence.
        --sam            -    SAM is provided instead of MHAP. The reads file will be ignored, and seq and qual fields
                              from the SAM file will be used. [false]
        --mhap           -    Overlaps are in PAF format instead of MHAP. [false]

  Algorithm:
        --bq            FLT   Threshold for the average base quality of the input reads. If a read has average BQ <
                              specified, the read will be skipped. If valu

In [9]:
%%bash
time \
/usr/local/bioinformatics/racon-v0.5.0/bin/racon \
20170911_oly_pacbio_cat.fastq \
20170911_minimap2_pacbio_oly.paf \
20170918_oly_pacbio_miniasm_reads.gfa \
20170918_oly_pacbio_racon_consensus.fasta

[18:20:50 main] Using PAF for input alignments. (20170911_minimap2_pacbio_oly.paf)
[18:20:50 main] Loading reads.
[18:23:58 main] Hashing qnames.
[18:24:05 main] Parsing the overlaps file.
[18:24:05 main] Unique overlaps will be filtered on the fly.
[Mon, 18 Sep 17 18:24:05 +0000 ERROR] #5: Unexpected value found! Could not find qname 'm170301_162825_42134_c101174162550000001823269408211762_s1_p0/27749/11100_22781' in the input contigs file! Exiting.In function: 'ParsePAF'.

[Mon, 18 Sep 17 18:24:05 +0000 ERROR] #5: Unexpected value found! Could not find qname 'm170301_162825_42134_c101174162550000001823269408211762_s1_p0/27749/11100_22781' in the input contigs file! Exiting.In function: 'ParsePAF'.
Exiting.

real	3m16.449s
user	2m28.040s
sys	0m17.592s


#### Reading through the [example on the Racon GitHub page](https://github.com/isovic/racon#consensus-from-mappingsoverlaps-in-pafmhap-format), it seems like I need to do another step with [Minimap](https://github.com/lh3/minimap2) before I can run Racon.

#### Using the default settings in Minimap will map the reads to contigs, which is needed for Racon.

In [10]:
%%bash
time \
/usr/local/bioinformatics/minimap2-2.1.1_x64-linux/minimap2 \
-t 23 \
20170911_oly_pacbio_cat.fastq 20170905_minimap2_pacibio_oly.paf > 20170918_minimap2_mapping_oly_pacbio.paf

[M::mm_idx_gen::116.151*1.50] collected minimizers
[M::mm_idx_gen::120.734*2.20] sorted minimizers
[M::main::120.735*2.20] loaded/built the index for 537523 target sequence(s)
[M::mm_mapopt_update::129.279*2.12] mid_occ = 288; max_occ = 1715
[M::mm_idx_stat] kmer size: 15; skip: 10; is_HPC: 0; #seq: 537523
[M::mm_idx_stat::131.206*2.11] distinct minimizers: 153278982 (34.77% are singletons); average occurrences: 4.884; average spacing: 5.344
[M::mm_idx_gen::263.975*1.78] collected minimizers
[M::mm_idx_gen::267.588*2.05] sorted minimizers
[M::main::267.588*2.05] loaded/built the index for 537296 target sequence(s)
[M::mm_mapopt_update::278.744*2.01] mid_occ = 260; max_occ = 1515
[M::mm_idx_stat] kmer size: 15; skip: 10; is_HPC: 0; #seq: 537296
[M::mm_idx_stat::280.708*2.01] distinct minimizers: 155901188 (34.32% are singletons); average occurrences: 4.800; average spacing: 5.345
[M::mm_idx_gen::412.938*1.82] collected minimizers
[M::mm_idx_gen::416.895*2.00] sorted minimizers
[M::main:

#### Presumbly that worked. Let's run Racon...

In [11]:
%%bash
time \
/usr/local/bioinformatics/racon-v0.5.0/bin/racon -t 24\
20170911_oly_pacbio_cat.fastq \
20170918_minimap2_mapping_oly_pacbio.paf \
20170918_oly_pacbio_miniasm_reads.gfa \
20170918_oly_pacbio_racon_consensus.fasta

ERROR: Not all required arguments specified. Parameter 'reads' missing a value. Exiting.

real	0m0.064s
user	0m0.000s
sys	0m0.000s


In [12]:
%%bash
time \
/usr/local/bioinformatics/racon-v0.5.0/bin/racon -t 24 \
20170911_oly_pacbio_cat.fastq \
20170918_minimap2_mapping_oly_pacbio.paf \
20170918_oly_pacbio_miniasm_reads.gfa \
20170918_oly_pacbio_racon_consensus.fasta

[19:36:47 main] Using PAF for input alignments. (20170918_minimap2_mapping_oly_pacbio.paf)
[19:36:47 main] Loading reads.
[19:39:18 main] Hashing qnames.
[19:39:22 main] Parsing the overlaps file.
[19:39:22 main] Unique overlaps will be filtered on the fly.
[19:39:22 main] Overlaps will be fully aligned.
[19:39:22 ConsensusFromOverlaps] Running consensus.
[19:39:22 ConsensusFromOverlaps] Separating overlaps to individual contigs.
[19:39:22 ConsensusFromOverlaps] In total, there are 2898 contigs for consensus, each containing:
[19:39:22 ConsensusFromOverlaps] 	[0] utg000001l 0 alignments, contig len: 12432
[19:39:22 ConsensusFromOverlaps] 	[1] utg000002l 0 alignments, contig len: 15812
[19:39:22 ConsensusFromOverlaps] 	[2] utg000003l 0 alignments, contig len: 41979
[19:39:22 ConsensusFromOverlaps] 	[3] utg000004l 0 alignments, contig len: 24099
[19:39:22 ConsensusFromOverlaps] 	[4] utg000005l 0 alignments, contig len: 14902
[19:39:22 ConsensusFromOverlaps] 	[5] utg000006l 0 alignments, 

In [13]:
%%bash
ls -lh /home/data/20170918_oly_pacbio_racon_consensus.fasta

-rw-r--r-- 1 root root 0 Sep 18 19:39 /home/data/20170918_oly_pacbio_racon_consensus.fasta


Looks like it's empty. Let's check

In [14]:
%%bash
head /home/data/20170918_oly_pacbio_racon_consensus.fasta

#### Well, that's no good. Time to figure out what's wrong...

#### The solution is possibly related to the mapping step (performed with minimap above). I think this [GitHub issue](https://github.com/isovic/racon/issues/18) points out that the GFA needs to be converted to FASTA in order to be used for mapping with minimap. The issue provides the following awk command to perform this conversion. Let's try it out.

In [15]:
%%bash
head /home/data/20170918_oly_pacbio_miniasm_reads.gfa

S	utg000001l	GTGTGTGTGGTGTTGGATATTTGGATGGGCATCTATTTTTCATTTTAAAATGCAAAGATTTTCTCCATTGATTTGAAAACAAATATATACACAGCTTCAGCAAATATATCACTAGAAGTGTTCCATTCGGTTTTGTCTGTGTAACTTCGCAGCACAGTTAAGAGCTGAGTAGGGGCGCTCTCCTAATAATCTATTTTTAGAATGTGTATATACATCAAAACACCAAAACGAACAAAATAATCGAACACTAGCAAATATCTGTAATTTAGAATAATTAATGTTTATTTAATGCACGGAAGTATATACACACAAAAAACTTACATTAAACGGCTCTTGTTGGTCGTGAGTGTAGTCTGACGCCCTCAGATATCTAAACAACAACATCGTATCGCGATGCACTTTCATTATCATGACGTCATCAACGATCTAGTGTGCCGCATTTCGCGAAGATCTAGAGTGGCGGATCAATTGTTTTTTTGTCAACGAAAAAATTACTTTAGATGTAATGCAAATCATGATTGGAAATATTAAAAGACAATTAATTTGAGCAACAATGCCACCCGTTATAAAATTCATAACATTATCTATCTTATCCCTAGGAAAGCATTGATGCCATATTTATTTAAAAAAATAAGGGGGGGGATATGTTGAGGCATGGGTGGGTCATGCATATCGTTCATTGCACAAGGCTGCCCTAAATTGCTGTTTAAACGCGAAACCAAATAATATATTAAGCTATTTGAAAAACGTTTGTTACTCCTTGATGAATTTCTTAGAGAAATAATTGTCTATTTGAAATCGAGGTGCAGATGACTGGGTAAGATAGTTTCAACATACGATACAATATATTATTTTAACAATTATAAGTCAAAATCTGTAAAGGAATGCAAACGTAATCATAAACTACAGAGGTTTATAAAAACATTCATGAATCAAAAGTTAGATTTCAATTCCCAGTATGCCCTTAATATTGAAATCTATTGATAA

In [16]:
%%bash
awk '$1 ~/S/ {print ">"$2"\n"$3}' 20170918_oly_pacbio_miniasm_reads.gfa > 20170918_oly_pacbio_miniasm_reads.fasta 

In [17]:
%%bash
head 20170918_oly_pacbio_miniasm_reads.fasta

>utg000001l
GTGTGTGTGGTGTTGGATATTTGGATGGGCATCTATTTTTCATTTTAAAATGCAAAGATTTTCTCCATTGATTTGAAAACAAATATATACACAGCTTCAGCAAATATATCACTAGAAGTGTTCCATTCGGTTTTGTCTGTGTAACTTCGCAGCACAGTTAAGAGCTGAGTAGGGGCGCTCTCCTAATAATCTATTTTTAGAATGTGTATATACATCAAAACACCAAAACGAACAAAATAATCGAACACTAGCAAATATCTGTAATTTAGAATAATTAATGTTTATTTAATGCACGGAAGTATATACACACAAAAAACTTACATTAAACGGCTCTTGTTGGTCGTGAGTGTAGTCTGACGCCCTCAGATATCTAAACAACAACATCGTATCGCGATGCACTTTCATTATCATGACGTCATCAACGATCTAGTGTGCCGCATTTCGCGAAGATCTAGAGTGGCGGATCAATTGTTTTTTTGTCAACGAAAAAATTACTTTAGATGTAATGCAAATCATGATTGGAAATATTAAAAGACAATTAATTTGAGCAACAATGCCACCCGTTATAAAATTCATAACATTATCTATCTTATCCCTAGGAAAGCATTGATGCCATATTTATTTAAAAAAATAAGGGGGGGGATATGTTGAGGCATGGGTGGGTCATGCATATCGTTCATTGCACAAGGCTGCCCTAAATTGCTGTTTAAACGCGAAACCAAATAATATATTAAGCTATTTGAAAAACGTTTGTTACTCCTTGATGAATTTCTTAGAGAAATAATTGTCTATTTGAAATCGAGGTGCAGATGACTGGGTAAGATAGTTTCAACATACGATACAATATATTATTTTAACAATTATAAGTCAAAATCTGTAAAGGAATGCAAACGTAATCATAAACTACAGAGGTTTATAAAAACATTCATGAATCAAAAGTTAGATTTCAATTCCCAGTATGCCCTTAATATTGAAATCTATTGATAAT

The conversion seems to have worked. Let's try minimap again.

In [18]:
%%bash
time \
/usr/local/bioinformatics/minimap2-2.1.1_x64-linux/minimap2 \
-t 23 \
20170918_oly_pacbio_miniasm_reads.fasta 20170905_minimap2_pacibio_oly.paf > 20170918_minimap2_mapping_fasta_oly_pacbio.paf

[M::mm_idx_gen::1.485*1.00] collected minimizers
[M::mm_idx_gen::1.814*1.54] sorted minimizers
[M::main::1.814*1.54] loaded/built the index for 2898 target sequence(s)
[M::mm_mapopt_update::1.991*1.49] mid_occ = 33; max_occ = 163
[M::mm_idx_stat] kmer size: 15; skip: 10; is_HPC: 0; #seq: 2898
[M::mm_idx_stat::2.052*1.48] distinct minimizers: 5673441 (90.70% are singletons); average occurrences: 1.168; average spacing: 5.332
[M::main] Version: 2.1.1-r341
[M::main] CMD: /usr/local/bioinformatics/minimap2-2.1.1_x64-linux/minimap2 -t 23 20170918_oly_pacbio_miniasm_reads.fasta 20170905_minimap2_pacibio_oly.paf
[M::main] Real time: 2.069 sec; CPU: 3.048 sec

real	0m2.126s
user	0m2.528s
sys	0m0.532s


In [19]:
%%bash
time \
/usr/local/bioinformatics/racon-v0.5.0/bin/racon -t 24 \
20170911_oly_pacbio_cat.fastq \
20170918_minimap2_mapping_fasta_oly_pacbio.paf \
20170918_oly_pacbio_miniasm_reads.gfa \
20170918_oly_pacbio_racon_consensus.fasta

[20:19:06 main] Using PAF for input alignments. (20170918_minimap2_mapping_fasta_oly_pacbio.paf)
[20:19:06 main] Loading reads.
[20:21:30 main] Hashing qnames.
[20:21:33 main] Parsing the overlaps file.
[20:21:33 main] Unique overlaps will be filtered on the fly.
[20:21:33 main] Overlaps will be fully aligned.
[20:21:33 ConsensusFromOverlaps] Running consensus.
[20:21:33 ConsensusFromOverlaps] Separating overlaps to individual contigs.
[20:21:33 ConsensusFromOverlaps] In total, there are 2898 contigs for consensus, each containing:
[20:21:33 ConsensusFromOverlaps] 	[0] utg000001l 0 alignments, contig len: 12432
[20:21:33 ConsensusFromOverlaps] 	[1] utg000002l 0 alignments, contig len: 15812
[20:21:33 ConsensusFromOverlaps] 	[2] utg000003l 0 alignments, contig len: 41979
[20:21:33 ConsensusFromOverlaps] 	[3] utg000004l 0 alignments, contig len: 24099
[20:21:33 ConsensusFromOverlaps] 	[4] utg000005l 0 alignments, contig len: 14902
[20:21:33 ConsensusFromOverlaps] 	[5] utg000006l 0 alignm

In [20]:
%%bash
ls -lh /home/data/20170918_oly_pacbio_racon_consensus.fasta

-rw-r--r-- 1 root root 0 Sep 18 20:21 /home/data/20170918_oly_pacbio_racon_consensus.fasta


Still empty!! What is wrong here? Going to give this whole thing a redo

In [21]:
%%bash
time /usr/local/bioinformatics/minimap2-2.1.1_x64-linux/minimap2 -x ava-pb -t 24 \
20170911_oly_pacbio_cat.fastq \
20170911_oly_pacbio_cat.fastq \
| gzip -1 > 20170918_minimap2_overlaps_pacbio_oly.paf.gz

time /usr/local/bioinformatics/miniasm-0.2/miniasm -f \
/home/data/20170911_oly_pacbio_cat.fastq \
/home/data/20170918_minimap2_overlaps_pacbio_oly.paf.gz > \
/home/data/20170918_oly_pacbio_miniasm_assembly.gfa

awk '$1 ~/S/ {print ">"$2"\n"$3}' 20170918_oly_pacbio_miniasm_assembly.gfa > 20170918_oly_pacbio_miniasm_assembly.fasta 

time /usr/local/bioinformatics/minimap2-2.1.1_x64-linux/minimap2 \
20170918_oly_pacbio_miniasm_assembly.fasta \
20170911_oly_pacbio_cat.fastq > \
20170918_oly_pacbio_minimap_mappings.paf

time /usr/local/bioinformatics/racon-v0.5.0/bin/racon -t24 \
20170911_oly_pacbio_cat.fastq \
20170918_oly_pacbio_minimap_mappings.paf \
20170918_oly_pacbio_miniasm_assembly.gfa \
20170918_oly_pacbio_racon1_consensus.fasta

[M::mm_idx_gen::163.071*1.41] collected minimizers
[M::mm_idx_gen::170.048*2.13] sorted minimizers
[M::main::170.048*2.13] loaded/built the index for 537516 target sequence(s)
[M::mm_mapopt_update::191.298*2.00] mid_occ = 216; max_occ = 1259
[M::mm_idx_stat] kmer size: 19; skip: 5; is_HPC: 1; #seq: 537516
[M::mm_idx_stat::195.252*1.98] distinct minimizers: 263663554 (42.00% are singletons); average occurrences: 3.579; average spacing: 4.239
[M::worker_pipeline::245.393*5.81] mapped 78070 sequences
[M::worker_pipeline::279.888*7.95] mapped 77485 sequences
[M::worker_pipeline::295.217*8.67] mapped 75465 sequences
[M::worker_pipeline::306.062*9.17] mapped 61141 sequences
[M::worker_pipeline::326.860*10.08] mapped 61571 sequences
[M::worker_pipeline::344.395*10.75] mapped 61587 sequences
[M::worker_pipeline::361.010*11.34] mapped 61048 sequences
[M::worker_pipeline::378.873*11.92] mapped 61149 sequences
[M::worker_pipeline::392.906*12.34] mapped 60782 sequences
[M::worker_pipeline::406.847

In [22]:
%%bash
time /usr/local/bioinformatics/racon-v0.5.0/bin/racon -t24 \
20170911_oly_pacbio_cat.fastq \
20170918_oly_pacbio_minimap_mappings.paf \
20170918_oly_pacbio_miniasm_assembly.gfa \
20170918_oly_pacbio_racon1_consensus.fasta

ERROR: Unknown parameter '-t24'.

real	0m0.002s
user	0m0.000s
sys	0m0.000s


In [23]:
%%bash
time /usr/local/bioinformatics/racon-v0.5.0/bin/racon -t 24 \
20170911_oly_pacbio_cat.fastq \
20170918_oly_pacbio_minimap_mappings.paf \
20170918_oly_pacbio_miniasm_assembly.gfa \
20170918_oly_pacbio_racon1_consensus.fasta

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


Well, that's annoying! Let's see if we actually got an output file this time!

In [24]:
%%bash
ls -lh 20170918_oly_pacbio_racon1_consensus.fasta

-rw-r--r-- 1 root root 33M Sep 18 22:19 20170918_oly_pacbio_racon1_consensus.fasta


Let's count how many contigs were generated.

In [25]:
%%bash
time grep -c '>' 20170918_oly_pacbio_racon1_consensus.fasta

2896



real	0m0.137s
user	0m0.004s
sys	0m0.004s


In [1]:
%%bash
date

Fri Sep 29 20:00:52 UTC 2017


In [2]:
%%bash
mkdir /home/owl/Athaliana/201709_oly_pacbio_assembly_minimap_asm_racon

In [3]:
ls

170210_PCB-CC_MS_EEE_20kb_P6v2_D01_1_filtered_subreads.fasta
170210_PCB-CC_MS_EEE_20kb_P6v2_D01_1_filtered_subreads.fastq*
170228_PCB-CC_AL_20kb_P6v2_C01_1_filtered_subreads.fasta
170228_PCB-CC_AL_20kb_P6v2_C01_1_filtered_subreads.fastq*
170228_PCB-CC_AL_20kb_P6v2_D01_1_filtered_subreads.fastq*
170228_PCB-CC_AL_20kb_P6v2_E01_1_filtered_subreads.fastq*
170307_PCB-CC_AL_20kb_P6v2_C01_1_filtered_subreads.fastq*
170307_PCB-CC_AL_20kb_P6v2_C02_1_filtered_subreads.fastq*
170314_PCB-CC_20kb_P6v2_A01_1_filtered_subreads.fastq*
170314_PCB-CC_20kb_P6v2_A02_1_filtered_subreads.fastq*
170314_PCB-CC_20kb_P6v2_A03_1_filtered_subreads.fastq*
170314_PCB-CC_20kb_P6v2_A04_1_filtered_subreads.fastq*
20170905_minimap2_pacibio_oly.paf
20170907_docker_pacbio_oly_minimap2.ipynb
20170911_minimap2_pacbio_oly.paf
20170911_minimap2_pacbio_oly_170210_fq.paf
20170911_minimap2_pacibio_oly_170210.paf
20170911_minimap2_pacibio_oly_170210_vs_170228C01.paf
20170911_oly_pacbio_cat.fastq
20170918_docker_pacbio_oly_minias

In [1]:
%%bash
time cp 201709* /home/owl/Athaliana/201709_oly_pacbio_assembly_minimap_asm_racon/


real	25m36.294s
user	0m0.368s
sys	1m21.724s


In [2]:
%%bash
ls -lh /home/owl/Athaliana/201709_oly_pacbio_assembly_minimap_asm_racon/

total 73G
-rw-r--r-- 1 1027 users    0 Sep 29 21:29 20170905_minimap2_pacibio_oly.paf
-rw-r--r-- 1 1027 users  44K Sep 29 21:29 20170907_docker_pacbio_oly_minimap2.ipynb
-rw-r--r-- 1 1027 users  40G Sep 29 21:40 20170911_minimap2_pacbio_oly.paf
-rw-r--r-- 1 1027 users 1.3G Sep 29 21:41 20170911_minimap2_pacbio_oly_170210_fq.paf
-rw-r--r-- 1 1027 users 1.3G Sep 29 21:42 20170911_minimap2_pacibio_oly_170210.paf
-rw-r--r-- 1 1027 users    0 Sep 29 21:42 20170911_minimap2_pacibio_oly_170210_vs_170228C01.paf
-rw-r--r-- 1 1027 users  24G Sep 29 21:51 20170911_oly_pacbio_cat.fastq
-rw-r--r-- 1 1027 users  45K Sep 29 21:51 20170918_docker_pacbio_oly_miniasm0.2.ipynb
-rw-r--r-- 1 1027 users 1.3M Sep 29 21:51 20170918_docker_pacbio_oly_racon0.5.0.ipynb
-rw-r--r-- 1 1027 users    0 Sep 29 21:51 20170918_minimap2_mapping_fasta_oly_pacbio.paf
-rw-r--r-- 1 1027 users    0 Sep 29 21:51 20170918_minimap2_mapping_oly_pacbio.paf
-rw-r--r-- 1 1027 users 6.6G Sep 29 21:54 20170918_minimap2_overlaps_pacbio